## Installation

<https://python.langchain.com/docs/get_started/quickstart>

In [ ]:
!pip install langchain
!pip install openai
!pip install "langserve[all]"

## 设置环境变量

In [24]:
import os
os.environ["OPENAI_API_KEY"]= "sk-..."
os.environ["HTTPS_PROXY"]= "http://127.0.0.1:7890"
os.environ["OPENAI_API_KEY"]

'sk-...'

## Building with LangChain

LangChain提供了许多可用于构建语言模型应用程序的模块。

最简单、最常见的链包含：

LLM/Chat Model: 语言模型是这里的核心推理引擎。

Prompt Template: 为语言模型提供指令。

Output Parser: 将来自语言模型的原始响应转换为更可用的格式。

## LLM / Chat Model

有两种语言模型：

- LLM: 底层模型将字符串作为输入并返回字符串

- ChatModel: 底层模型将消息列表作为输入并返回消息

基本消息由 BaseMessage 定义

- content: 消息的内容
- role: BaseMessage 所来自的实体。

LangChain提供了几个对象来方便区分不同的角色：

- HumanMessage
- AIMessage
- SystemMessage
- FunctionMessage/ToolMessage

使用 .invoke() 调用 LLM 或 ChatModel

In [13]:
from langchain.llms import OpenAI
from langchain.chat_models import ChatOpenAI
from langchain.schema import HumanMessage

llm = OpenAI()
chat_model = ChatOpenAI()

text = "What would be a good company name for a company that makes colorful socks?"
messages = [HumanMessage(content=text)]

print(llm.invoke(text))

chat_model.invoke(messages)



Colorful Feet Co.


AIMessage(content='ColorfulStep')

## Prompt templates 提示词模版

In [10]:
from langchain.prompts import PromptTemplate

prompt = PromptTemplate.from_template("What is a good name for a company that makes {product}?")
prompt.format(product='colorful socks')

'What is a good name for a company that makes colorful socks?'

In [11]:
from langchain.prompts.chat import ChatPromptTemplate

template = "You are a helpful assistant that translates {input_language} to {output_language}."
human_template = "{text}"

chat_prompt = ChatPromptTemplate.from_messages([
    ("system", template),                     
    ("human", human_template),
])
chat_prompt.format_messages(input_language="English", output_language="French", text="I love programming.")

[SystemMessage(content='You are a helpful assistant that translates English to French.'),
 HumanMessage(content='I love programming.')]

## Output parsers 输出解析器

In [12]:
from langchain.schema import BaseOutputParser

class CommaSeparatedListOutputParser(BaseOutputParser):
    """Parse the output of an LLM call to a comma-separated list."""


    def parse(self, text: str):
        """Parse the output of an LLM call."""
        return text.strip().split(", ")

CommaSeparatedListOutputParser().parse("hi, bye")

['hi', 'bye']

## Composing with LCEL 用 LCEL 组合

现在可以将所有这些组合成一条链。

In [14]:
from typing import List

from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.schema import BaseOutputParser

class CommaSeparatedListOutputParser(BaseOutputParser):
    """Parse the output of an LLM call to a comma-separated list."""

    def parse(self, text: str) -> List[str]:
        """Parse the output of an LLM call."""
        return text.strip().split(", ")
    
template = """You are a helpful assistant who generates comma separated lists.
A user will pass in a category, and you should generate 5 objects in that category in a comma separated list.
ONLY return a comma separated list, and nothing more."""
human_template = "{text}"

chat_prompt = ChatPromptTemplate.from_messages([
    ("system", template),                     
    ("human", human_template),
])
chain = chat_prompt | ChatOpenAI() | CommaSeparatedListOutputParser()
chain.invoke({"text": "colors"})

# 使用 | 语法将这些组件连接在一起。| 语法由 LangChain 表达式语言（LCEL）提供支持

['red', 'blue', 'green', 'yellow', 'purple']

## Serving with LangServe 使用 LangServe 提供服务

In [18]:
!pip install fastapi

Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.1/92.1 kB 4.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 80.9/80.9 kB 1.7 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.0/67.0 kB 5.1 MB/s eta 0:00:00
  Attempting uninstall: anyio
    Found existing installation: anyio 4.2.0
    Uninstalling anyio-4.2.0:
      Successfully uninstalled anyio-4.2.0


## Playground

```bash
python 1.serve.py
```

<http://localhost:8000/category_chain/playground/>

## Client

In [25]:
from langserve import RemoteRunnable

remote_chain = RemoteRunnable("http://localhost:8000/category_chain/")
remote_chain.invoke({"text": "colors"})

['red', 'blue', 'green', 'yellow', 'purple']